In [1]:
import numpy as np
import xgboost as xgb
import pandas as pd
import category_encoders as ce
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

In [3]:
train_data = pd.read_csv('../data/LOANS_TRAIN.csv')
test_data = pd.read_csv('../data/LOANS_TEST.csv')

train_data.drop(columns=['id','grade', 'emp_title', 'title'])
test_data.drop(columns=['id','grade', 'emp_title', 'title'])

#Strip percent(%) from int_rate
train_data['int_rate'] = train_data['int_rate'].str.rstrip('%').astype(float)
test_data['int_rate'] = test_data['int_rate'].str.rstrip('%').astype(float)

#Strip percent(%) from revol_util
train_data['revol_util'] = train_data['revol_util'].str.rstrip('%').astype(float)
test_data['revol_util'] = test_data['revol_util'].str.rstrip('%').astype(float)

X_train = train_data.iloc[:,:-1]
y_train = train_data.iloc[:,-1]
X_test = test_data.iloc[:,:]
y_test = test_data.iloc[:,-1]
print(y_test)
# ce_OHE = ce.OneHotEncoder(cols=['loan_status', 'application_type', 'home_ownership', 'verification_status'])
# train_data = ce_OHE.fit_transform(train_data)

# ce_BE = ce.BinaryEncoder(cols=['loan_status'])
# train_data = ce_BE.fit_transform(train_data)
# ce_LOO = ce.LeaveOneOutEncoder(cols=['grade', 'emp_title', 'issue_d', 'addr_state', 'purpose', 'initial_list_status', 'title', 'application_type', 'home_ownership', 'verification_status', 'sub_grade', 'zip_code', 'emp_length', 'mort_acc', 'earliest_cr_line'])
# train_X = ce_LOO.fit_transform(X_train, y_train)

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
42475    0.0
42476    0.0
42477    0.0
42478    0.0
42479    0.0
Name: pub_rec_bankruptcies, Length: 42480, dtype: float64


In [22]:
def convert_to_numeric(df):
    X = df.select_dtypes(include=[np.object])
    #For loop to loop one by one with col type object
    for col in X.columns:
        X[col]=pd.Categorical(X[col],categories=df[col].dropna().unique())
        #creating dummy variable
        X_col = pd.get_dummies(X[col])
        X = X.drop(col,axis=1)
        X_col.columns = X_col.columns.tolist()
        frames = [X_col, X] 
        X = pd.concat(frames,axis=1)
    return X
X_train_numeric = convert_to_numeric(X_train)
y_train_numeric = convert_to_numeric(y_train)
X_test_numeric = convert_to_numeric(X_test)
y_test_numeric = convert_to_numeric(y_test)

<ipython-input-22-11914985a863>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col]=pd.Categorical(X[col],categories=df[col].dropna().unique())


In [ ]:
def auc2(m, train, test): 
    return (metrics.roc_auc_score(y_train,m.predict(train)),
                            metrics.roc_auc_score(y_test,m.predict(test)))

lg = lgb.LGBMClassifier(silent=False)
param_dist = {"max_depth": [25,50, 75],
              "learning_rate" : [0.01,0.05,0.1],
              "num_leaves": [300,900,1200],
              "n_estimators": [200]
             }
grid_search = GridSearchCV(lg, n_jobs=-1, param_grid=param_dist, cv = 3, scoring="roc_auc", verbose=5)
grid_search.fit(X_train_numeric,y_train_numeric)
grid_search.best_estimator_

d_train = lgb.Dataset(X_train_numeric, label=y_train_numeric)
params = {"max_depth": 50, "learning_rate" : 0.1, "num_leaves": 900,  "n_estimators": 300}

# # Without Categorical Features
# model2 = lgb.train(params, d_train)
# auc2(model2, X_train, X_train)

#With Catgeorical Features
cate_features_name = ["subgrade","emp_length","home_ownership","verification_status","issue_d",
                 "purpose","zipcode","addr_state", "earliest_cr_line", "initial_list_status",
                 "application_type", "loan_status"]
model2 = lgb.train(params, d_train, categorical_feature = cate_features_name)
auc2(model2, X_test_numeric, y_test_numeric)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


/Users/Rohan/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan]
  warnings.warn(


ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in the following fields: grade, sub_grade, emp_title, emp_length, home_ownership, verification_status, issue_d, purpose, title, zip_code, addr_state, earliest_cr_line, initial_list_status, application_type

<ipython-input-5-dc2f71e5faf5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col]=pd.Categorical(X[col],categories=train_X[col].dropna().unique())


TypeError: '<' not supported between instances of 'float' and 'str'

In [ ]:

# clf = xgb.XGBClassifier(
#     tree_method="gpu_hist", enable_categorical=True, use_label_encoder=False
# )
# # X is the dataframe we created in previous snippet
# clf.fit(X, y)

KeyError: 'cat_feature'